In [5]:
%use lets-plot, krangl, kotlin-statistics, klaxon

In [156]:
import java.net.URL
fun DataFrame.Companion.fromJsonObject(url: String, mapper: (JsonObject) -> JsonArray<*>): DataFrame {
    val uri = URL(url).toURI().toURL()
    val obj = Parser.default().parse(uri.openStream()) as JsonObject
    val arr = obj.let(mapper).toJsonString()
    return DataFrame.fromJsonString(arr)
}

fun <R> DataFrame.mapStringColumn(column: String, default: R, mapFunc: (String) -> R?) =
    addColumn(column) { it[column].map<String> { i -> mapFunc(i) ?: default } }
infix fun DataCol.notEq(i: Any) = !eq(i)

In [168]:
val dataUrl = "https://data.taipei/api/v1/dataset/42254d50-7eff-48bd-870f-5257fc4c1928?scope=resourceAquire"
val df = DataFrame.fromJsonObject(dataUrl) { it.obj("result")!!.array<Any?>("results")!! }
    .selectIf { it.name != "_id" }
    .groupBy("縣市別")
    .mapStringColumn("百分比[%]", 0.0, String::toDoubleOrNull)
    .mapStringColumn("交易量[公斤]", 0, String::toIntOrNull)
    .rename("百分比[%]" to "百分比")
df.schema()

DataFrame with 585 observations
縣市別      [Str]  總計, 總計, 總計, 總計, 總計, 總計, 總計, 總計, 總計, 總計, 總計, 總計, 總計, 總計, 總計, 總計, 總計, 總計, 總計, 總計, 總計, 總計, 總計, 總計, 總計, ...
年別       [Str]  64年, 65年, 66年, 67年, 68年, 69年, 70年, 71年, 72年, 73年, 74年, 75年, 76年, 77年, 78年, 79年, 80年, 81年, 82年, 83年, ...
交易量[公斤]  [Int]  221671801, 203142953, 202221342, 227717267, 226001772, 241992043, 236540160, 271459190, 281120311, 2...
百分比      [Dbl]  100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, ...


In [195]:
lets_plot(df.filter{ it["縣市別"] eq "總計" }.toMap()) +
    geom_line { x = "年別"; y = "交易量[公斤]" } +
    ggtitle("歷年臺北市蔬菜交易總量")

In [166]:
lets_plot(df.filter { it["縣市別"] notEq "總計" }.toMap()) +
    geom_bar() { x = "年別"; weight = "百分比"; y = "百分比";  fill = "縣市別" } +
    ggtitle("歷年臺北市蔬菜交易縣市比例")

In [184]:
lets_plot(df.filter { it["縣市別"] notEq "總計" }.toMap()) + ggtitle("") +
    geom_point(size = 1.5) { x = "百分比"; y = "交易量[公斤]"; color = "縣市別" }

In [196]:
lets_plot(df.filter { it["縣市別"] notEq "總計" }.toMap()) + ggtitle("") +
    geom_bar { x = "年別"; weight = "交易量[公斤]"; fill = "縣市別" }

In [203]:
lets_plot(df.filter { it["縣市別"] notEq "總計" }.toMap()) + ggtitle("") +
    geom_crossbar() { x = "縣市別";  }